# Paper results
### Updates from the last version (`obsolete/paper_figures.ipynb`):
1. Use Gibbs
2. Use data for rh bc
3. Use identity parameterization for a (enable positive and negative values of a)
4. Use the mapping `var(a) = 2*var(sqrt(a))**2` for the variance of a
5. Use higher resolution for the mesh

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipyplot
from advection_diffusion_inference_utils import (read_experiment_data,
                                                 build_grids,
                                                 plot_time_series,
                                                 read_data_files)
import cuqi


In [2]:
def matplotlib_setup(SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE):
    plt.rc('font', size=MEDIUM_SIZE)         # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title 

## Cases:

**note**:

In all the cases
- noise level: Gibbs
- Sampler: NUTS 
- Number of samples:

**cases**:


1. CA (diffusion only)
    - 1.1. synthetic (m1-l) : 4 cases
        - constant/constant
        - constant/varying
        - varying/constant
        - varying/varying
    - 1.2. real (all mice, all ears) : 10 cases
        - varying


2. CA-ST (diffusion only)
    - 2.1 synthetic (m1-l) : 10 case
        - varying
    - 2.2 real (all mice, all ears) : 10 cases
        - varying

3. CA (advection-diffusion)
    - 3.1 synthetic (m1-l) : 2 cases
        - varying/small_a/varying
        - varying/large_a/varying
    - 3.2 real (all mice, all ears) : 10 cases
        - varying

Total number of cases: 4+ 10+ 10+ 10+ 2+ 10 = 46

Number of runs: 6 major runs: 

| Case | Number of runs | label | notes: | running? |
| --- | --- | --- | --- | --- |
| <font color='black'>1.1 </font>  | 4  | paper_v2_CASynthDiff    |  |  |
| <font color='black'>1.2 </font>  | 10 | paper_v2_CARealDiff     |  |  |
| <font color='black'>2.1 </font>  | 10 | paper_v2_CASTSynthDiff  |  |  |
| <font color='black'>2.2 </font>  | 10 | paper_v2_CASTRealDiff   |  |  |
| <font color='black'>3.1 </font>  | 2  | paper_v2_CASynthAdvDiff |  |  |
| <font color='black'>3.2 </font>  | 10 | paper_v2_CARealAdvDiff  |  |  |



### Error in estimation: (only heterogenous diffusivity and heterogenous inference)

Synth:
| Case | Error in data | Recon. error (diffusivity) | Recon. error (advection) | STD norm |
| --- | --- | --- | --- | --- |
| CA (diffusion only) | - | - | - | - |
| CA-ST (diffusion only)(avg mice) |  |  |  |  |
| CA-ST (diffusion only)(avg max mice) |  |  |  |  |
| CA-ST (diffusion only)(avg min mice) |  |  |  |  |
| CA (advection-diffusion) (small a) |  |  |  |  |
| CA (advection-diffusion) (large a) |  |  |  |  |


Real:

| Case | Error in data | STD norm |
| --- | --- | --- |
| CA (diffusion only)(avg mice)       |  |  |
| CA (diffusion only)(max mice)       |  |  |
| CA (diffusion only)(min mice)       |  |  |
| CA-ST (diffusion only)(avg mice)    |  |  |
| CA-ST (diffusion only)(max mice)    |  |  |
| CA-ST (diffusion only)(min mice)    |  |  |
| CA (advection-diffusion) (avg mice) |  |  |
| CA (advection-diffusion) (max mice) |  |  |
| CA (advection-diffusion) (min mice) |  |  |


Numbers shortened 
| Case | Error in data recon. | STD norm |
| --- | --- | --- |
| CA (diffusion only)(avg mice)       |  |  |
| CA (diffusion only)(max mice)       |  |  |
| CA (diffusion only)(min mice)       |  |  |
| CA-ST (diffusion only)(avg mice)    |  |  |
| CA-ST (diffusion only)(max mice)    |  |  |
| CA-ST (diffusion only)(min mice)    |  |  |
| CA (advection-diffusion) (avg mice) |  |  |
| CA (advection-diffusion) (max mice) |  |  |
| CA (advection-diffusion) (min mice) |  |  |

```
min 
max 
avg 

** eleminate the cases that are not convergent **
** make sure the two for CA have the same cases and the same cases only **

paper_v2_CARealDiff
data error list







```





- define error in data
- define error in reconstruction for both diffusivity and advection
- overlay advection and diffusion based diffusivity CI


## 1. CA (diffusion only)

### 1.1. Synthetic (m1-l) (4 cases) (constant/constant, constant/varying, varying/constant, varying/varying)

In [3]:

paper_v2_CASynthDiff_tags =\
["m1_l_NUTS_constant_100.0_syntheticFromDiffusion_constant_3000_fromDataAvg_paper_v2_CASynthDiff__0_5_0.1_zero",
"m1_l_NUTS_constant_100.0_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASynthDiff__0_5_0.1_zero",
"m1_l_NUTS_smooth_50_200_syntheticFromDiffusion_constant_3000_fromDataAvg_paper_v2_CASynthDiff__0_5_0.1_zero",
"m1_l_NUTS_smooth_50_200_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASynthDiff__0_5_0.1_zero"]

paper_v2_CASynthDiff_dir = ["../../../Collab-BrainEfflux-Data/after_UQIPI24/results9"]*len(paper_v2_CASynthDiff_tags)


Plot the experiments:

In [ ]:
# read the data
data_paper_v2_CASynthDiff = [read_experiment_data(paper_v2_CASynthDiff_dir[i], paper_v2_CASynthDiff_tags[i]) for i in range(len(paper_v2_CASynthDiff_tags))]

In [5]:
# extract the samples and exact values, etc
samples_list = []
exact_list = []
locations_list = []
ess_list = []
data_list = []
mean_recon_data_list = []
times_list = []
exact_data_list = []
c_geom_list = []
L_list = []
short_tag = []
for j, data_dict in enumerate(data_paper_v2_CASynthDiff):
    samples_list.append(data_dict['samples'])
    ess_list.append(samples_list[-1].compute_ess())
    exact_list.append(data_dict['exact'])
    locations_list.append(data_dict['locations'])
    data_list.append(data_dict['data'])
    mean_recon_data_list.append(data_dict['mean_recon_data'])
    times_list.append(data_dict['times'])
    exact_data_list.append(data_dict['exact_data'])
    short_tag.append(paper_v2_CASynthDiff_tags[j].split('_')[0] + ' ' + paper_v2_CASynthDiff_tags[j].split('_')[1])
    # WARNING MAKE SURE THIS FACTOR IS CONSISTENT 
    factor = 1.2
    L_list.append(locations_list[-1][-1] * factor)


<div style="border: 2px solid #FFB74D; background-color: #FFF3E0; border-radius: 8px; padding: 10px; font-family: Arial, sans-serif; color: #333; box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1); max-width: 750px; margin: 0 auto;">
  <strong style="color: #E65100;">⚠️ Note:</strong> 
Note: the varying in space results are not convergent
</div>

In [ ]:
# Figure 1
matplotlib_setup(12, 14, 16)





# Create figure
figure, axs = plt.subplots(2,3, figsize=(12, 6.5*2.2/3))
#---------------------- time series for the constant real parameter case
idx00 = 0
plt.sca(axs[0, 0])
plot_time_series(times_list[idx00], locations_list[idx00], data_list[idx00])

# increase spacing between subplots
figure.subplots_adjust(hspace=0.4, wspace=0.4)

#---------------------- 95% CI (constant inference)
plt.sca(axs[0, 1])
idx01 = 0
exact =  cuqi.array.CUQIarray(exact_list[idx01][0],is_par=False, geometry=samples_list[idx01].geometry)
samples_list[idx01]
samples_list[idx01].plot_ci(exact = exact)
#plt.ylim([70, 150])
# set ticks labels
plt.gca().set_xticklabels(["$x_0$"])
plt.xlabel('unknown parameter')
plt.ylabel('$c^2$')

#---------------------- 95% CI (variable inference)

idx02 = 1
plt.sca(axs[0, 2])
samples_list[idx02].plot_ci(exact = exact_list[idx02])
#plt.ylim([70, 150])
# set ticks labels
plt.xlabel('$\\xi$')
plt.ylabel('$c^2$')
# plot vertical dotted lines for locations
for loc in locations_list[idx02]:
    plt.axvline(x = loc, color = 'gray', linestyle = '--')
#---------------------- time series for the variable real parameter case
# Load data

plt.sca(axs[1, 0])
idx10 = 2
plot_time_series(times_list[idx10], locations_list[idx10], data_list[idx10])


#---------------------- 95% CI (constant inference)
plt.sca(axs[1, 1])
idx11 = 2
exact =  cuqi.array.CUQIarray(exact_list[idx11][0],is_par=False, geometry=samples_list[idx11].geometry)
samples_list[idx11].plot_ci(exact = exact)

# set ticks labels
plt.gca().set_xticklabels(["$x_0$"])
plt.xlabel('unknown parameter')
plt.ylabel('$c^2$')




#---------------------- 95% CI (variable inference)

plt.sca(axs[1, 2])
idx12 = 3

samples_list[idx12].plot_ci(95, exact = exact_list[idx12])


plt.xlabel('$\\xi$')
plt.ylabel('$c^2$')
# change legend of last plot
plt.legend(['95% CI' ,'Mean', 'Exact', ])
# plot vertical dotted lines for locations
for loc in locations_list[idx12]:
    plt.axvline(x = loc, color = 'gray', linestyle = '--')




In [ ]:
samples_list[idx12].compute_ess()

### 1.2. Real (all mice, all ears) (8~10 cases) (varying)

In [9]:


paper_v2_CARealDiff_tags =[
"m1_l_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m1_r_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m2_l_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m2_r_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m3_l_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m3_r_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m4_l_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m4_r_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m6_l_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero",
"m6_r_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CARealDiff__0_5_0.1_zero"]
paper_v2_CARealDiff_dir = ['../../../Collab-BrainEfflux-Data/after_UQIPI24/results11/']*len(paper_v2_CARealDiff_tags)

In [ ]:
# Figures list
paper_v2_CARealDiff_figs = [paper_v2_CARealDiff_dir[i] +'output'+paper_v2_CARealDiff_tags[i] + '/experiment_' + paper_v2_CARealDiff_tags[i] + '.png' for i in range(len(paper_v2_CARealDiff_tags))]

ipyplot.plot_images(paper_v2_CARealDiff_figs, img_width=800/2)

In [ ]:
# read the data
paper_v2_CARealDiff_dict_list = []
for i in range(len(paper_v2_CARealDiff_tags)):
    print(i)
    paper_v2_CARealDiff_dict_list.append(read_experiment_data(paper_v2_CARealDiff_dir[i], paper_v2_CARealDiff_tags[i]))

In [ ]:
# extract the samples and exact values, etc
samples_list_CARealDiff = []
exact_list = []
locations_list_CARealDiff = []
ess_list = []
data_list = []
mean_recon_data_list_CARealDiff = []
times_list_CARealDiff = []
exact_data_list = []
c_geom_list = []
L_list = []
short_tag = []
data_error_list_paper_v2_CARealDiff = []
std_over_data_list_paper_v2_CARealDiff = []
new_ess_list = []

for j, data_dict in enumerate(paper_v2_CARealDiff_dict_list):
    samples_list_CARealDiff.append(data_dict['samples'])
    ess_list.append(samples_list_CARealDiff[-1].compute_ess())
    exact_list.append(data_dict['exact'])
    locations_list_CARealDiff.append(data_dict['locations'])
    data_list.append(data_dict['data'])
    mean_recon_data_list_CARealDiff.append(data_dict['mean_recon_data'])
    times_list_CARealDiff.append(data_dict['times'])
    exact_data_list.append(data_dict['exact_data'])
    short_tag.append(paper_v2_CARealDiff_tags[j].split('_')[0] + ' ' + paper_v2_CARealDiff_tags[j].split('_')[1])
    # WARNING MAKE SURE THIS FACTOR IS CONSISTENT 
    factor = 1.2
    L_list.append(locations_list_CARealDiff[-1][-1] * factor)
    data_error_list_paper_v2_CARealDiff.append(np.linalg.norm(data_list[-1]-mean_recon_data_list_CARealDiff[-1])/ np.linalg.norm(data_list[-1]))
    # read data STD
    real_times, real_locations, real_data, real_std_data = read_data_files(data_dict['experiment_par'])
    std_over_data_list_paper_v2_CARealDiff.append(np.linalg.norm(np.linalg.norm(real_std_data)/np.linalg.norm(real_data)))
    new_ess_list.append(ess_list[-1])



# remove idx (-1, -3, -5)
data_error_list_paper_v2_CARealDiff = np.delete(data_error_list_paper_v2_CARealDiff, [-1, -3, -5])
std_over_data_list_paper_v2_CARealDiff = np.delete(std_over_data_list_paper_v2_CARealDiff, [-1, -3, -5])
new_ess_list = [new_ess_list[i] for i in range(len(new_ess_list)) if i not in [9, 7, 5]]

print(data_dict['experiment_par'].version)
# print min, max, avg
print('data error list')
print(data_error_list_paper_v2_CARealDiff)
print('min', np.min(data_error_list_paper_v2_CARealDiff))
print('max', np.max(data_error_list_paper_v2_CARealDiff))
print('avg', np.mean(data_error_list_paper_v2_CARealDiff))

# print min, max, avg
print('std_over_data_list')
print(std_over_data_list_paper_v2_CARealDiff)
print('min', np.min(std_over_data_list_paper_v2_CARealDiff))
print('max', np.max(std_over_data_list_paper_v2_CARealDiff))
print('avg', np.mean(std_over_data_list_paper_v2_CARealDiff))
print('#################')
print(new_ess_list)
print(np.mean(new_ess_list))
print(np.min(new_ess_list))


    

In [ ]:
len(new_ess_list)


In [ ]:
ess_list[0]

In [ ]:
# Create a plot of 9 panels
#first row: noisy data, reconstruction, diffusion CI (large a)
#second row: noisy data, reconstruction, diffusion CI (small a)
#third row: PDF of advection (both) , ess (both) , empty

matplotlib_setup(12, 14, 16)




#Create a 3 by 3 grid of plots
# make 2 subplots to draw the legend in the bottom



fig= plt.figure( figsize=(12, 6.5))
lines_list = []
subfigs = fig.subfigures(2, 1, height_ratios=[4,1])

####################################################
idx1_CARealDiff = 0
axs = subfigs[0].subplots(3, 3 , gridspec_kw=dict( bottom=0.1, hspace=0.5))
plt.sca(axs[0, 0])

lines_list.append(plot_time_series(times=times_list_CARealDiff[idx1_CARealDiff],
                 data=data_list[idx1_CARealDiff],
                 locations=locations_list_CARealDiff[idx1_CARealDiff],
                 plot_legend=False))
plt.ylabel('Concentration \n $u$')
# add mouse and ear
plt.title('data (noisy) '+ short_tag[idx1_CARealDiff])
# turn of ticks
plt.xticks([])
plt.xlabel('')

plt.sca(axs[1, 0])
lines = plot_time_series(times=times_list_CARealDiff[idx1_CARealDiff],
                 data=mean_recon_data_list_CARealDiff[idx1_CARealDiff],
                 locations=locations_list_CARealDiff[idx1_CARealDiff],
                 plot_legend=False)
plt.ylabel('Concentration \n $u$')
plt.title('mean reconstruction')
plt.gca().xaxis.set_label_coords(0.5, 0.2)

plt.sca(axs[2,0])
fig_ifo = samples_list_CARealDiff[idx1_CARealDiff].plot_ci()
plt.xlim(0, L_list[idx1_CARealDiff])
plt.ylabel('diffusivity\n $c^2$')


####################################################
idx2_CARealDiff = 2


plt.sca(axs[0, 1])
lines_list.append(plot_time_series(times=times_list_CARealDiff[idx2_CARealDiff],
                 data=data_list[idx2_CARealDiff],
                 locations=locations_list_CARealDiff[idx2_CARealDiff],
                 plot_legend=False))
plt.ylabel('')
# turn of ticks
plt.xticks([])
plt.xlabel('')
plt.title('data (noisy) '+ short_tag[idx2_CARealDiff])


plt.sca(axs[1, 1])
lines = plot_time_series(times=times_list_CARealDiff[idx2_CARealDiff],
                 data=mean_recon_data_list_CARealDiff[idx2_CARealDiff],
                 locations=locations_list_CARealDiff[idx2_CARealDiff],
                 plot_legend=False)
plt.ylabel('')
plt.title('mean reconstruction')
plt.gca().xaxis.set_label_coords(0.5, 0.2)

plt.sca(axs[2,1])
fig_ifo = samples_list_CARealDiff[idx2_CARealDiff].plot_ci()
plt.xlim(0, L_list[idx2_CARealDiff])
# legend off
plt.legend().set_visible(False)


####################################################
idx3_CARealDiff = 4

plt.sca(axs[0, 2])

lines_list.append(plot_time_series(times=times_list_CARealDiff[idx3_CARealDiff],
                 data=data_list[idx3_CARealDiff],
                 locations=locations_list_CARealDiff[idx3_CARealDiff],
                 plot_legend=False))
plt.ylabel('')
# turn of ticks
plt.xticks([])
plt.xlabel('')
plt.title('data (noisy) '+ short_tag[idx3_CARealDiff])


plt.sca(axs[1, 2])
lines = plot_time_series(times=times_list_CARealDiff[idx3_CARealDiff],
                 data=mean_recon_data_list_CARealDiff[idx3_CARealDiff],
                 locations=locations_list_CARealDiff[idx3_CARealDiff],
                 plot_legend=False)
plt.ylabel('')
plt.title('mean reconstruction')
# xlabel position up a bit

plt.gca().xaxis.set_label_coords(0.5, 0.2)

plt.sca(axs[2,2])
fig_ifo = samples_list_CARealDiff[idx3_CARealDiff].plot_ci()
plt.xlim(0, L_list[idx3_CARealDiff])
# legend off
plt.legend().set_visible(False)

plt.tight_layout()

# third column





# plot legend
axLegend = subfigs[1].subplots(1, 3)
plt.sca(axLegend[0])
plt.axis('off')
plt.legend(lines_list[0][0], lines_list[0][1], ncol=1, loc='center', bbox_to_anchor=(0.5, 0.5))
# ["{:.0f}".format(location_i) for location_i in locations_list[0]], loc='center', bbox_to_anchor=(0.5, 0.5))

plt.sca(axLegend[1])
plt.axis('off')
plt.legend(lines_list[1][0], lines_list[1][1], ncol=1, loc='center', bbox_to_anchor=(0.5, 0.5))

plt.sca(axLegend[2])
plt.axis('off')
plt.legend(lines_list[2][0], lines_list[2][1], ncol=1, loc='center', bbox_to_anchor=(0.5, 0.5))

# print locations formated 2 space after decimal point

print(["{:.0f}".format(location_i) for location_i in locations_list_CARealDiff[idx1_CARealDiff]])
#plt.tight_layout()
print(lines)

## 2. CA-ST (diffusion only)

### 2.1 synthetic (m1-l) : 1 case (varying)


1. Read data files

In [19]:
list_CASTSynthDiff_tags = ["m1_l_NUTS_sampleMean_m1:l:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASTSynthDiff__4_5_0.1_zero",
                           "m1_r_NUTS_sampleMean_m1:r:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASTSynthDiff__4_5_0.1_zero",
                           "m3_l_NUTS_sampleMean_m3:l:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASTSynthDiff__4_5_0.1_zero",
                           "m3_r_NUTS_sampleMean_m3:r:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASTSynthDiff__4_5_0.1_zero",
                           "m4_l_NUTS_sampleMean_m4:l:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASTSynthDiff__4_5_0.1_zero",
                           "m4_r_NUTS_sampleMean_m4:r:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASTSynthDiff__4_5_0.1_zero",
                           "m6_l_NUTS_sampleMean_m6:l:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASTSynthDiff__4_5_0.1_zero",
                           "m6_r_NUTS_sampleMean_m6:r:NUTS:constant:100.0:real:heterogeneous:1000:0.1:v:April22:2024:a::4:5_syntheticFromDiffusion_heterogeneous_3000_fromDataAvg_paper_v2_CASTSynthDiff__4_5_0.1_zero"
]
list_CASTSynthDiff = ['../../../Collab-BrainEfflux-Data/after_UQIPI24/results13/']*len(list_CASTSynthDiff_tags)

In [ ]:
# plots
list_CASTSynthDiff_figs = [list_CASTSynthDiff[i] +'output'+list_CASTSynthDiff_tags[i] + '/experiment_' + list_CASTSynthDiff_tags[i] + '.png' for i in range(len(list_CASTSynthDiff_tags))]

ipyplot.plot_images(list_CASTSynthDiff_figs, img_width=800)

In [ ]:


# read the data
data_dict_list = []
for i in range(len(list_CASTSynthDiff)):
    data_dict_list.append(read_experiment_data(list_CASTSynthDiff[i], list_CASTSynthDiff_tags[i]))



In [ ]:
matplotlib_setup(12, 14, 16)

# powerpoint slide size in inches: 13.333 x 7.5 (widescreeen)
# obtain samples and plot the ci
samples_list = []
exact_list = []
locations_list = []
ess_list = []
# error in the data
exact_recon_data_error_CAST_synth = []
exact_recon_param_error_CAST_synth = []
noise_in_data_CAST_synth = []
recon_error_list = []
exact_data_list = []
mean_recon_data_list = []
for data_dict in data_dict_list:
    samples_list.append(data_dict['samples'])
    ess_list.append(samples_list[-1].compute_ess())
    exact_list.append(data_dict['exact'])
    locations_list.append(data_dict['locations'])
    exact_data_list.append(data_dict['exact_data'])
    mean_recon_data_list.append(data_dict['mean_recon_data'])
    # error in the data
    exact_recon_data_error_CAST_synth.append(np.linalg.norm(data_dict['exact_data'] - data_dict['mean_recon_data'].flatten())/np.linalg.norm(data_dict['exact_data']))
    exact_recon_param_error_CAST_synth.append(np.linalg.norm(data_dict['exact'] - data_dict['samples'].funvals.mean())/np.linalg.norm(data_dict['exact']))
    noise_in_data_CAST_synth.append(np.linalg.norm(data_dict['data'].flatten() - data_dict['exact_data'])/np.linalg.norm(data_dict['data']))


#Create a 3 by 3 grid of plots
fig, axs = plt.subplots(3, 3, figsize=(12, 6.5))
#Plot the first 7 samples in
for i in range(3):
    for j in range(3):
        idx = 3*i+j
        if idx == len(samples_list):
            # clear the axis
            axs[i, j].axis('off')
        elif idx > len(samples_list): 
            # plot legend
            plt.sca(axs[i, j])
            axs[i, j].axis('off')
            plt.legend(fig_ifo, ['Mean', 'Exact', '95% CI'], loc='lower center', bbox_to_anchor=(0.8, -0.2))
        else:
        
            samples = samples_list[idx]
            exact = exact_list[idx]
            plt.sca(axs[i, j])
            fig_ifo = samples.plot_ci(exact=exact)
            # plot vertical gray lines at the locations
            for location in locations_list[idx]:
                plt.axvline(x=location, color='gray', linestyle='--', alpha=0.5)
            # turn off the legend
            plt.legend().set_visible(False)
            # turn off x-axis label for all figures except image 5,6,7
            # which mouse, which ear
            plt.title(
                list_CASTSynthDiff_tags[idx].split('_')[0] + ' ' + list_CASTSynthDiff_tags[idx].split('_')[1],
                x=0.5, y=0.82)

# remove y ticks 
# add x label for (1,1), (1,2), (2,1)
axs[1, 1].set_xlabel('location')
axs[1, 2].set_xlabel('location')
axs[2, 0].set_xlabel('location')

# y label for (0,0), (1,0), (2,0)
axs[0, 0].set_ylabel('$c^2$')
axs[1, 0].set_ylabel('$c^2$')
axs[2, 0].set_ylabel('$c^2$')

# tighten the figure
plt.tight_layout()
# remove white space
plt.subplots_adjust(wspace=0.2, hspace=0.15)


In [ ]:
#min max avg

print('min ess:', np.min(ess_list))
print('max ess:', np.max(ess_list))
print('avg ess:', np.mean(ess_list))

###  2.2 real (all mice, all ears) : 8 cases (varying)

In [28]:
list_CASTRealDiff_tags = [
                            "m3_l_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CASTRealDiff__4_5_0.1_zero",
                            "m3_r_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CASTRealDiff__4_5_0.1_zero",
                            "m4_r_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CASTRealDiff__4_5_0.1_zero",
                            "m6_l_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CASTRealDiff__4_5_0.1_zero",
                            "m6_r_NUTS_constant_100.0_real_heterogeneous_3000_fromDataAvg_paper_v2_CASTRealDiff__4_5_0.1_zero"]


list_CASTRealDiff = ['../../../Collab-BrainEfflux-Data/after_UQIPI24/results15/']*len(list_CASTRealDiff_tags)

In [ ]:
# plots
list_CASTRealDiff_figs = [list_CASTRealDiff[i] +'output'+list_CASTRealDiff_tags[i] + '/experiment_' + list_CASTRealDiff_tags[i] + '.png' for i in range(len(list_CASTRealDiff_tags))]

ipyplot.plot_images(list_CASTRealDiff_figs, img_width=800)

In [ ]:
# read the data
CASTRealDiff_dict_list = []
for i in range(len(list_CASTRealDiff)):
    CASTRealDiff_dict_list.append(read_experiment_data(list_CASTRealDiff[i], list_CASTRealDiff_tags[i]))

In [ ]:
# powerpoint slide size in inches: 13.333 x 7.5 (widescreeen)
# obtain samples and plot the ci
samples_list = []
exact_list = []
locations_list = []
ess_list = []
data_error_list_CAST_real = []
std_over_data_list_CAST_real = []
mean_recon_data_list = []
data_list = []
for data_dict in CASTRealDiff_dict_list:
    samples_list.append(data_dict['samples'])
    ess_list.append(samples_list[-1].compute_ess())
    exact_list.append(data_dict['exact'])
    locations_list.append(data_dict['locations'])
    mean_recon_data_list.append(data_dict['mean_recon_data'])
    data_list.append(data_dict['data'])
    data_error_list_CAST_real.append(np.linalg.norm(data_list[-1]-mean_recon_data_list[-1])/ np.linalg.norm(data_list[-1]))
    # read data STD
    real_times, real_locations, real_data, real_std_data = read_data_files(data_dict['experiment_par'])
    std_over_data_list_CAST_real.append(np.linalg.norm(np.linalg.norm(real_std_data)/np.linalg.norm(real_data)))

In [ ]:
# pri#print(data_error_list_paper_v2_CARealDiff)
print(data_dict['experiment_par'].version)
# print min, max, avg
print('data error list')
print('min', np.min(data_error_list_CAST_real))
print('max', np.max(data_error_list_CAST_real))
print('avg', np.mean(data_error_list_CAST_real))
#print(std_over_data_list_paper_v2_CARealDiff)
# print min, max, avg
print('std_over_data_list')
print('min', np.min(std_over_data_list_CAST_real))
print('max', np.max(std_over_data_list_CAST_real))
print('avg', np.mean(std_over_data_list_CAST_real))

# print ess
print('min ess:', np.min(ess_list))
print('max ess:', np.max(ess_list))
print('avg ess:', np.mean(ess_list))
print(ess_list)
print('#################')

In [ ]:
matplotlib_setup(12, 14, 16)




#Create a 3 by 3 grid of plots
fig, axs = plt.subplots(3, 2, figsize=(9, 6.5))
#Plot the first 7 samples in
for i in range(3):
    for j in range(2):
        idx = 2*i+j
        if idx < len(samples_list):

        
            samples = samples_list[idx]
            exact = exact_list[idx]
            plt.sca(axs[i, j])
            fig_ifo = samples.plot_ci()
            # plot vertical gray lines at the locations
            for location in locations_list[idx]:
                plt.axvline(x=location, color='gray', linestyle='--', alpha=0.5)
            # turn off the legend
            if idx != 5:
                plt.legend().set_visible(False)
            else:
                plt.legend([fig_ifo[0], fig_ifo[2]], ['Mean', '95% CI'])#, loc='lower center', bbox_to_anchor=(0.8, -0.2))
            # turn off x-axis label for all figures except image 5,6,7
            # which mouse, which ear
            plt.title(
                list_CASTSynthDiff_tags[idx].split('_')[0] + ' ' + list_CASTSynthDiff_tags[idx].split('_')[1],
                x=0.5, y=0.82)

# remove y ticks 
# add x label for (1,1), (1,2), (2,1)
axs[2, 0].set_xlabel('location')
axs[2, 1].set_xlabel('location')


# y label for (0,0), (1,0), (2,0)
axs[0, 0].set_ylabel('$c^2$')
axs[1, 0].set_ylabel('$c^2$')
axs[2, 0].set_ylabel('$c^2$')

# tighten the figure
plt.tight_layout()
# remove white space
plt.subplots_adjust(wspace=0.2, hspace=0.15)


In [ ]:
fig_ifo
# print min, max, avg, ess
print('min ess:', np.min(ess_list))
print('max ess:', np.max(ess_list))
print('avg ess:', np.mean(ess_list))



## 3. CA (advection-diffusion)

### 3.1 synthetic (m1-l) : 2 cases (varying/small_a/varying, varying/large_a/varying)

In [36]:
paper_v2_CASynthAdvDiff_dir = ['../../../Collab-BrainEfflux-Data/after_UQIPI24/results8/']*2
paper_v2_CASynthAdvDiff_tags = ['m1_l_NUTS_custom_1_100.0_syntheticFromDiffusion_advection_diffusion_3000_fromDataAvg_paper_v2_CASynthAdvDiff__0_5_0.9_zero',
                              'm1_l_NUTS_custom_1_100.0_syntheticFromDiffusion_advection_diffusion_3000_fromDataAvg_paper_v2_CASynthAdvDiff__0_5_0.1_zero']
 

In [ ]:
# plots
paper_v2_CASynthAdvDiff_figs = [paper_v2_CASynthAdvDiff_dir[i] +'output'+paper_v2_CASynthAdvDiff_tags[i] + '/experiment_' + paper_v2_CASynthAdvDiff_tags[i] + '.png' for i in range(len(paper_v2_CASynthAdvDiff_tags))]

ipyplot.plot_images(paper_v2_CASynthAdvDiff_figs, img_width=800/2)

In [ ]:
#read the data files
paper_v2_CASynthAdvDiff_dict_list = []
for i in range(len(paper_v2_CASynthAdvDiff_dir)):
    paper_v2_CASynthAdvDiff_dict_list.append(read_experiment_data(paper_v2_CASynthAdvDiff_dir[i], paper_v2_CASynthAdvDiff_tags[i]))

In [39]:
# extract the samples and exact values, etc
samples_list = []
paper_v2_CASynthAdvDiff_exact_list = []
locations_list = []
ess_list = []
data_list = []
mean_recon_data_list = []
times_list = []
exact_data_list = []
for data_dict in paper_v2_CASynthAdvDiff_dict_list:
    samples_list.append(data_dict['samples'])
    ess_list.append(samples_list[-1].compute_ess())
    paper_v2_CASynthAdvDiff_exact_list.append(data_dict['exact'])
    locations_list.append(data_dict['locations'])
    data_list.append(data_dict['data'])
    mean_recon_data_list.append(data_dict['mean_recon_data'])
    times_list.append(data_dict['times'])
    exact_data_list.append(data_dict['exact_data'])

In [ ]:
# Create a plot of 9 panels
#first row: noisy data, reconstruction, diffusion CI (large a)
#second row: noisy data, reconstruction, diffusion CI (small a)
#third row: PDF of advection (both) , ess (both) , empty

matplotlib_setup(12, 14, 16)




#Create a 3 by 3 grid of plots
# make 2 subplots to draw the legend in the bottom



fig= plt.figure( figsize=(9, 6.5))

subfigs = fig.subfigures(2, 1, height_ratios=[4,1])

axs = subfigs[0].subplots(3, 2 , gridspec_kw=dict( bottom=0.1, hspace=0.3))
plt.sca(axs[0, 0])
lines = plot_time_series(times=times_list[0],
                 data=exact_data_list[0].reshape(data_list[0].shape),
                 locations=locations_list[0],
                 plot_legend=False)
plt.ylabel('Concentration \n $u$')
plt.title('data (exact)')
# turn of ticks
plt.xticks([])
plt.xlabel('')

plt.sca(axs[1, 0])
lines = plot_time_series(times=times_list[0],
                 data=data_list[0],
                 locations=locations_list[0],
                 plot_legend=False)
plt.ylabel('Concentration \n $u$')
plt.title('data (noisy)')
# turn of ticks
plt.xticks([])
plt.xlabel('')

plt.sca(axs[2, 0])
lines = plot_time_series(times=times_list[0],
                 data=mean_recon_data_list[0],
                 locations=locations_list[0],
                 plot_legend=False)
plt.ylabel('Concentration \n $u$')
plt.title('mean reconstruction')


plt.sca(axs[0, 1])
lines = plot_time_series(times=times_list[1],
                 data=exact_data_list[1].reshape(data_list[1].shape),
                 locations=locations_list[1],
                 plot_legend=False)
plt.ylabel('')
# turn of ticks
plt.xticks([])
plt.xlabel('')
plt.title('data (exact)')

plt.sca(axs[1, 1])
lines = plot_time_series(times=times_list[1],
                 data=data_list[1],
                 locations=locations_list[1],
                 plot_legend=False)
plt.ylabel('')
# turn of ticks
plt.xticks([])
plt.xlabel('')
plt.title('data (noisy)')


plt.sca(axs[2, 1])
lines = plot_time_series(times=times_list[1],
                 data=mean_recon_data_list[1],
                 locations=locations_list[1],
                 plot_legend=False)
plt.ylabel('')
plt.title('mean reconstruction')

plt.tight_layout()

# plot legend
axLegend = subfigs[1].subplots(1, 1)
plt.sca(axLegend)
plt.axis('off')
plt.legend(lines[0], lines[1], ncol=3, loc='center', bbox_to_anchor=(0.5, 0.5))
# ["{:.0f}".format(location_i) for location_i in locations_list[0]], loc='center', bbox_to_anchor=(0.5, 0.5))


# print locations formated 2 space after decimal point

print(["{:.0f}".format(location_i) for location_i in locations_list[0]])
    
print(lines)


In [ ]:
# Generate a figure with 3 rows and 1 column, top row is ESS for both cases, second row is PDF for both cases, diffusivity CI for both cases

fig, axs = plt.subplots(2, 2, figsize=(8, 6.5*2.2/3), gridspec_kw={'hspace':0.35, 'wspace':0.35})

# plot ESS
plt.sca(axs[0,0])
plt.plot(ess_list[0], label='large $a$', color='blue')
plt.plot(ess_list[1], label='small $a$', color='orange')
plt.ylabel('ESS')
plt.xlabel('i')
plt.legend()

# plot PDF
map = samples_list[0].geometry.map
plt.sca(axs[1,0])
plt.hist(map(samples_list[0].samples[-1,:].flatten()), bins=50, alpha=0.5, label='large $a$', color='blue')
plt.hist(map(samples_list[1].samples[-1,:].flatten()), bins=50, alpha=0.5, label='small $a$', color='orange')
# plot scipy KDE
#import scipy.stats as sps
#kde = sps.gaussian_kde(map(samples_list[0].samples[-1,:].flatten()))
#x = np.linspace(-0.5, 5, 100)
#l1 = plt.plot(x, kde(x), color='blue', label='large $a$')
#kde = sps.gaussian_kde(map(samples_list[1].samples[-1,:].flatten()))
#l2 =plt.plot(x, kde(x), color='orange', label='small $a$')





#plt.ylim(0, 2.2)
plt.ylim(0, 1000)
plt.xlim(0, 5)
#plt.xlim(-0.5, 5)

# annotate the mean
plt.axvline(x=map(samples_list[0].samples[-1,:]).flatten().mean(),
             color='blue', linestyle='--', alpha=0.5)
plt.axvline(x=map(samples_list[1].samples[-1,:].flatten()).mean(),
            color='orange', linestyle='--', alpha=0.5)

# annotate the true value as points on the x-axis and make the point plotted over
#the x-axis
plt.plot([0.9], [0], '*', color='blue', label='0.9', clip_on=False)

plt.plot([0.1], [0], '*', color='orange', label='0.1', clip_on=False)



plt.ylabel('PDF')
plt.xlabel('advection speed $a$')
plt.xlabel('$a$')
plt.legend()

# plot diffusivity CI
plt.sca(axs[0,1])

print("WARNING L MIGHT NOT BE ACCURATE")
# PDE and coefficients grids
factor_L = 1.2 
L = locations_list[0][-1]*factor_L
coarsening_factor = 5
n_grid_c = 20


grid, grid_c, grid_c_fine, h, n_grid = build_grids(L, coarsening_factor, n_grid_c)
grid_c_geom = cuqi.geometry.MappedGeometry( cuqi.geometry.Continuous1D(grid_c), samples_list[0].geometry.map)
exact_new = cuqi.array.CUQIarray(paper_v2_CASynthAdvDiff_exact_list[0].to_numpy()[:-1], geometry= grid_c_geom)
fig_ifo = cuqi.samples.Samples(samples_list[0].samples[:-1,:], geometry=grid_c_geom).plot_ci(exact=exact_new)
#fig_ifo2 = cuqi.samples.Samples(samples_list[1].samples[:-1,:], geometry=grid_c_geom).plot_ci(plot_envelope_kwargs={'facecolor': 'orange'})
plt.xlim(0, L)

plt.legend(fig_ifo, ['Mean', 'Exact', '95% CI'])
plt.ylabel('diffusivity $c^2$')
plt.xlabel('location')
plt.title('large $a$\ncase', y=0.6, x=0.18)


plt.sca(axs[1,1])
grid, grid_c, grid_c_fine, h, n_grid = build_grids(L, coarsening_factor, n_grid_c)
grid_c_geom = cuqi.geometry.MappedGeometry( cuqi.geometry.Continuous1D(grid_c), samples_list[1].geometry.map)
exact_new = cuqi.array.CUQIarray(paper_v2_CASynthAdvDiff_exact_list[1].to_numpy()[:-1], geometry= grid_c_geom)
fig_ifo = cuqi.samples.Samples(samples_list[1].samples[:-1,:], geometry=grid_c_geom).plot_ci(exact=exact_new)
plt.xlim(0, L)
plt.legend(fig_ifo, ['Mean', 'Exact', '95% CI'])
plt.ylabel('diffusivity $c^2$')
plt.xlabel('location')
#plt.title('small $a$')
plt.title('small $a$\ncase', y=0.6, x=0.18)







<div style="border: 2px solid #FFB74D; background-color: #FFF3E0; border-radius: 8px; padding: 10px; font-family: Arial, sans-serif; color: #333; box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1); max-width: 750px; margin: 0 auto;">
  <strong style="color: #E65100;">⚠️ Note:</strong> 
  maybe correlation figure here
</div>

In [ ]:
# print min max ess, avg
print('min ess:', np.min(ess_list))
print('max ess:', np.max(ess_list))
print('avg ess:', np.mean(ess_list))
print(ess_list)

### 3.2 real (all mice, all ears) : 8 cases (varying)


In [ ]:

paper_v2_CARealAdvDiff_tags = ["m1_l_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero",
                             
"m1_r_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero",

"m2_l_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero",       
"m2_r_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero",
"m3_l_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero",   
"m4_l_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero",                                                                                                  
"m4_r_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero",       
"m6_l_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero",                                                                                                      
"m6_r_NUTS_custom_1_100.0_real_advection_diffusion_3000_fromDataAvg_paper_v2_CARealAdvDiff__0_5_0.1_zero" ]
paper_v2_CARealAdvDiff_dir = ['../../../Collab-BrainEfflux-Data/after_UQIPI24/results10/']*len(paper_v2_CARealAdvDiff_tags) 

len(paper_v2_CARealAdvDiff_tags)

In [ ]:
# plots
paper_v2_CARealAdvDiff_figs = [paper_v2_CARealAdvDiff_dir[i] +'output'+paper_v2_CARealAdvDiff_tags[i] + '/experiment_' + paper_v2_CARealAdvDiff_tags[i] + '.png' for i in range(len(paper_v2_CARealAdvDiff_tags))]

ipyplot.plot_images(paper_v2_CARealAdvDiff_figs, img_width=800/2)

In [ ]:
# read the data files
paper_v2_CARealAdvDiff_dict_list = []
for i in range(len(paper_v2_CARealAdvDiff_dir)):
    paper_v2_CARealAdvDiff_dict_list.append(read_experiment_data(paper_v2_CARealAdvDiff_dir[i], paper_v2_CARealAdvDiff_tags[i]))



In [ ]:
# extract the samples and exact values, etc
samples_list = []
exact_list = []
locations_list = []
ess_list = []
data_list = []
mean_recon_data_list = []
times_list = []
exact_data_list = []
c_geom_list = []
L_list = []
short_tag = []
data_error_list_paper_v2_CARealAdvDiff = []
std_over_data_list_paper_v2_CARealAdvDiff = []
new_ess_list = []
for j, data_dict in enumerate(paper_v2_CARealAdvDiff_dict_list):
    samples_list.append(data_dict['samples'])
    ess_list.append(samples_list[-1].compute_ess())
    exact_list.append(data_dict['exact'])
    locations_list.append(data_dict['locations'])
    data_list.append(data_dict['data'])
    mean_recon_data_list.append(data_dict['mean_recon_data'])
    times_list.append(data_dict['times'])
    exact_data_list.append(data_dict['exact_data'])
    factor_L = 1.2 
    L_list.append(locations_list[-1][-1]*factor_L)
    coarsening_factor = 5
    n_grid_c = 20
    grid, grid_c, grid_c_fine, h, n_grid = build_grids(L_list[-1], coarsening_factor, n_grid_c)
    c_geom_list.append(cuqi.geometry.MappedGeometry( cuqi.geometry.Continuous1D(grid_c), samples_list[0].geometry.map))
    short_tag.append(paper_v2_CARealAdvDiff_tags[j].split('_')[0] + ' ' + paper_v2_CARealAdvDiff_tags[j].split('_')[1])

    data_error_list_paper_v2_CARealAdvDiff.append(np.linalg.norm(data_list[-1]-mean_recon_data_list[-1])/ np.linalg.norm(data_list[-1]))
    # read data STD
    real_times, real_locations, real_data, real_std_data = read_data_files(data_dict['experiment_par'])
    std_over_data_list_paper_v2_CARealAdvDiff.append(np.linalg.norm(np.linalg.norm(real_std_data)/np.linalg.norm(real_data)))
    new_ess_list.append(ess_list[-1])

    
# remove idx (-1, -3, -5)
remove_idx = [6,8] #5,7,9
data_error_list_paper_v2_CARealAdvDiff = np.delete(data_error_list_paper_v2_CARealAdvDiff, remove_idx)
std_over_data_list_paper_v2_CARealAdvDiff = np.delete(std_over_data_list_paper_v2_CARealAdvDiff, remove_idx)
new_ess_list = [new_ess_list[i] for i in range(len(new_ess_list)) if i not in remove_idx]

print(data_dict['experiment_par'].version)
# print min, max, avg
print('data error list')
print(data_error_list_paper_v2_CARealAdvDiff)
print('min', np.min(data_error_list_paper_v2_CARealAdvDiff))
print('max', np.max(data_error_list_paper_v2_CARealAdvDiff))
print('avg', np.mean(data_error_list_paper_v2_CARealAdvDiff))

# print min, max, avg
print('std_over_data_list')
print(std_over_data_list_paper_v2_CARealAdvDiff)
print('min', np.min(std_over_data_list_paper_v2_CARealAdvDiff))
print('max', np.max(std_over_data_list_paper_v2_CARealAdvDiff))
print('avg', np.mean(std_over_data_list_paper_v2_CARealAdvDiff))
print('#################')
print(new_ess_list)
print(np.mean(new_ess_list))
print(np.min(new_ess_list))


In [ ]:
# pri#print(data_error_list_paper_v2_CARealDiff)
print(data_dict['experiment_par'].version)
# print min, max, avg
print('data error list')
print('min', np.min(data_error_list_paper_v2_CARealAdvDiff))
print('max', np.max(data_error_list_paper_v2_CARealAdvDiff))
print('avg', np.mean(data_error_list_paper_v2_CARealAdvDiff))
#print(std_over_data_list_paper_v2_CARealDiff)
# print min, max, avg
print('std_over_data_list')
print('min', np.min(std_over_data_list_paper_v2_CARealAdvDiff))
print('max', np.max(std_over_data_list_paper_v2_CARealAdvDiff))
print('avg', np.mean(std_over_data_list_paper_v2_CARealAdvDiff))

In [50]:
idx1_pre = 0
idx2 = 2
idx3 = 4


In [ ]:
# Create a plot of 9 panels
#first row: noisy data, reconstruction, diffusion CI (large a)
#second row: noisy data, reconstruction, diffusion CI (small a)
#third row: PDF of advection (both) , ess (both) , empty

matplotlib_setup(12, 14, 16)




#Create a 3 by 3 grid of plots
# make 2 subplots to draw the legend in the bottom



fig= plt.figure( figsize=(12, 8))
lines_list = []
subfigs = fig.subfigures(2, 1, height_ratios=[5,1])

####################################################

axs = subfigs[0].subplots(4, 3 , gridspec_kw=dict( bottom=0.1, hspace=0.5))
plt.sca(axs[0, 0])

lines_list.append(plot_time_series(times=times_list[idx1_pre],
                 data=data_list[idx1_pre],
                 locations=locations_list[idx1_pre],
                 plot_legend=False))
plt.ylabel('Concentration \n $u$')
# add mouse and ear
plt.title('data (noisy) '+ short_tag[idx1_pre])
# turn of ticks
plt.xticks([])
plt.xlabel('')

plt.sca(axs[1, 0])
plt.xticks([])
plt.xlabel('')
lines = plot_time_series(times=times_list[idx1_pre],
                 data=mean_recon_data_list[idx1_pre],
                 locations=locations_list[idx1_pre],
                 plot_legend=False)
plt.ylabel('Concentration \n $u$')
plt.title('predicted data (adv-diff)')
plt.gca().xaxis.set_label_coords(0.5, 0.2)

plt.sca(axs[2,0])
lines = plot_time_series(times=times_list_CARealDiff[idx1_CARealDiff],
                 data=mean_recon_data_list_CARealDiff[idx1_CARealDiff],
                 locations=locations_list_CARealDiff[idx1_CARealDiff],
                 plot_legend=False)

plt.ylabel('Concentration \n $u$')
plt.title('predicted data (diff)')
plt.gca().xaxis.set_label_coords(0.5, 0.2)

plt.sca(axs[3,0])
fig_ifo = cuqi.samples.Samples(samples_list[idx1_pre].samples[:-1,:], geometry=c_geom_list[idx1_pre]).plot_ci()
fig_ifo_CARealDiff = samples_list_CARealDiff[idx1_CARealDiff].plot_ci(plot_envelope_kwargs={'facecolor': 'g'}, color='g')

plt.xlim(0, L_list[idx1_pre])
plt.ylabel('diffusivity\n $c^2$')


plt.legend().set_bbox_to_anchor([1,-0.5])
####################################################



plt.sca(axs[0, 1])
lines_list.append(plot_time_series(times=times_list[idx2],
                 data=data_list[idx2],
                 locations=locations_list[idx2],
                 plot_legend=False))
plt.ylabel('')
# turn of ticks
plt.xticks([])
plt.xlabel('')
plt.title('data (noisy) '+ short_tag[idx2])


plt.sca(axs[1, 1])
lines = plot_time_series(times=times_list[idx2],
                 data=mean_recon_data_list[idx2],
                 locations=locations_list[idx2],
                 plot_legend=False)
plt.ylabel('')
plt.xticks([])
plt.xlabel('')
plt.title('predicted data (adv-diff)')
plt.gca().xaxis.set_label_coords(0.5, 0.2)

plt.sca(axs[2,1])
lines = plot_time_series(times=times_list_CARealDiff[idx2_CARealDiff],
                 data=mean_recon_data_list_CARealDiff[idx2_CARealDiff],
                 locations=locations_list_CARealDiff[idx2_CARealDiff],
                 plot_legend=False)
plt.ylabel('')
plt.title('predicted data (diff)')
plt.gca().xaxis.set_label_coords(0.5, 0.2)

plt.sca(axs[3,1])
fig_ifo = cuqi.samples.Samples(samples_list[idx2].samples[:-1,:], geometry=c_geom_list[idx2]).plot_ci()
plt.xlim(0, L_list[idx2])
fig_ifo_CARealDiff = samples_list_CARealDiff[idx2_CARealDiff].plot_ci(plot_envelope_kwargs={'facecolor': 'g'}, color='g')
# legend off
plt.legend().set_visible(False)


####################################################


plt.sca(axs[0, 2])

lines_list.append(plot_time_series(times=times_list[idx3],
                 data=data_list[idx3],
                 locations=locations_list[idx3],
                 plot_legend=False))
plt.ylabel('')
# turn of ticks
plt.xticks([])
plt.xlabel('')
plt.title('data (noisy) '+ short_tag[idx3])


plt.sca(axs[1, 2])
lines = plot_time_series(times=times_list[idx3],
                 data=mean_recon_data_list[idx3],
                 locations=locations_list[idx3],
                 plot_legend=False)
plt.ylabel('')
plt.xticks([])
plt.xlabel('')
plt.title('predicted data (adv-diff)')
# xlabel position up a bit

plt.gca().xaxis.set_label_coords(0.5, 0.2)


plt.sca(axs[2,2])
lines = plot_time_series(times=times_list_CARealDiff[idx3_CARealDiff],
                 data=mean_recon_data_list_CARealDiff[idx3_CARealDiff],
                 locations=locations_list_CARealDiff[idx3_CARealDiff],
                 plot_legend=False)
plt.ylabel('')
plt.title('predicted data (diff)')
plt.gca().xaxis.set_label_coords(0.5, 0.2)


plt.sca(axs[3,2])
fig_ifo = cuqi.samples.Samples(samples_list[idx3].samples[:-1,:], geometry=c_geom_list[idx3]).plot_ci()
plt.xlim(0, L_list[idx3])
fig_ifo_CARealDiff = samples_list_CARealDiff[idx3_CARealDiff].plot_ci(plot_envelope_kwargs={'facecolor': 'g'}, color='g')
# legend off
plt.legend().set_visible(False)

plt.tight_layout()

# third column





# plot legend
axLegend = subfigs[1].subplots(1, 3)
plt.sca(axLegend[0])
plt.axis('off')
plt.legend(lines_list[0][0], lines_list[0][1], ncol=1, loc='center', bbox_to_anchor=(0.5, 0.5))
# ["{:.0f}".format(location_i) for location_i in locations_list[0]], loc='center', bbox_to_anchor=(0.5, 0.5))

plt.sca(axLegend[1])
plt.axis('off')
plt.legend(lines_list[1][0], lines_list[1][1], ncol=1, loc='center', bbox_to_anchor=(0.5, 0.5))

plt.sca(axLegend[2])
plt.axis('off')
plt.legend(lines_list[2][0], lines_list[2][1], ncol=1, loc='center', bbox_to_anchor=(0.5, 0.5))

# print locations formated 2 space after decimal point

print(["{:.0f}".format(location_i) for location_i in locations_list[idx1_pre]])
#plt.tight_layout()
print(lines)

In [ ]:
# Generate a figure with 3 rows and 1 column, top row is ESS for both cases, second row is PDF for both cases, diffusivity CI for both cases

fig, axs = plt.subplots(3, 1, figsize=(4, 6.5), gridspec_kw={'hspace':0.35})


# plot ESS
plt.sca(axs[0])
color_list = ['blue', 'red', 'orange', 'brown', 'green', 'olive', 'purple']
idecs = [0, 1, 2, 3, 4, 6-1, 8-1]
for i, idx in enumerate(idecs):
    plt.plot(ess_list[idx], label=short_tag[idx], color=color_list[i])
#plt.plot(ess_list[idx1_pre], label=short_tag[idx1_pre], color='blue')
#plt.plot(ess_list[idx2], label=short_tag[idx2], color='orange')
#plt.plot(ess_list[idx3], label=short_tag[idx3], color='green')
plt.ylabel('ESS')
plt.xlabel('i')
#plt.legend()

# plot PDF KDE estimate
plt.sca(axs[1])
map = samples_list[idx1_pre].geometry.map
x = np.linspace(-0.5, 5, 100)
for i, idx in enumerate(idecs):
    
    #kde = sps.gaussian_kde(map(samples_list[idx].samples[-1,:].flatten()))
    #plt.plot(x, kde(x), color=color_list[i], label=short_tag[idx])
    plt.hist(map(samples_list[idx].samples[-1,:].flatten()), bins=50, alpha=0.3, label=short_tag[idx], color=color_list[i])

#plt.ylim(0, 2.2)
plt.xlim(0, 5)
plt.ylim(0, 1000)
plt.legend(ncol=2)

#plt.hist(map(samples_list[idx1_pre].samples[-1,:].flatten()), bins=50, alpha=0.5, label=short_tag[idx1_pre], color='blue')
#plt.hist(map(samples_list[idx2].samples[-1,:].flatten()), bins=50, alpha=0.5, label=short_tag[idx2], color='orange')
#plt.hist(map(samples_list[idx3].samples[-1,:].flatten()), bins=50, alpha=0.5, label=short_tag[idx3], color='green')
#plt.ylim(0, 800)
#plt.xlim(0, 8)
# annotate the mean
#plt.axvline(x=samples_list[0].samples[-1,:].mean(),
#             color='blue', linestyle='--')
#plt.axvline(x=samples_list[1].samples[-1,:].mean(),
#            color='orange', linestyle='--')
plt.ylabel('PDF')
#plt.xlabel('advection speed $a$')
plt.xlabel('$a$')
#plt.legend()

# Plot correlation between advection and diffusion
plt.sca(axs[2])

for i, idx in enumerate(idecs):
    # samples of average diffusion
    samples_diff_avg = np.array([np.average(samples_list[idx].samples[:-1,i]) for i in range(samples_list[idx].Ns)])
    # stack advection and diffusion
    diff_adv = cuqi.samples.Samples(np.vstack(
                                    (samples_diff_avg, samples_list[idx].samples[-1,:])))
    diff_adv.geometry = cuqi.geometry.MappedGeometry( cuqi.geometry.Discrete(['$c^2_\mathrm{avg}$', '$a$']), samples_list[0].geometry.map)

    # plot the correlation
    diff_adv.funvals.plot_pair(ax=plt.gca(), scatter_kwargs={'alpha':0.5, 'color':color_list[i]})

## First mouse
## samples of average diffusion
#samples_diff_avg = np.array([np.average(samples_list[idx1_pre].samples[:-1,i]) for i in range(samples_list[idx1_pre].Ns)])
## stack advection and diffusion
#diff_adv = cuqi.samples.Samples(np.vstack(
#                                (samples_diff_avg, samples_list[idx1_pre].samples[-1,:])))
#diff_adv.geometry = cuqi.geometry.MappedGeometry( cuqi.geometry.Discrete(['$c^2_\mathrm{avg}$', '$a$']), samples_list[0].geometry.map)
#
## plot the correlation
#diff_adv.funvals.plot_pair(ax=plt.gca(), scatter_kwargs={'alpha':0.5, 'color':'blue'})
#
## Second mouse
## samples of average diffusion
#samples_diff_avg = np.array([np.average(samples_list[idx2].samples[:-1,i]) for i in range(samples_list[idx2].Ns)])
## stack advection and diffusion
#diff_adv = cuqi.samples.Samples(np.vstack(
#                                (samples_diff_avg, samples_list[idx2].samples[-1,:])))
#diff_adv.geometry = cuqi.geometry.MappedGeometry( cuqi.geometry.Discrete(['$c^2_\mathrm{avg}$', '$a$']), samples_list[0].geometry.map)
#
## plot the correlation
#diff_adv.funvals.plot_pair(ax=plt.gca(), scatter_kwargs={'alpha':0.5, 'color':'orange'})
#
## Third mouse
## samples of average diffusion
#samples_diff_avg = np.array([np.average(samples_list[idx3].samples[:-1,i]) for i in range(samples_list[idx3].Ns)])
## stack advection and diffusion
#diff_adv = cuqi.samples.Samples(np.vstack(
#                                (samples_diff_avg, samples_list[idx3].samples[-1,:])))
#diff_adv.geometry = cuqi.geometry.MappedGeometry( cuqi.geometry.Discrete(['$c^2_\mathrm{avg}$', '$a$']), samples_list[0].geometry.map)
#
## plot the correlation
#diff_adv.funvals.plot_pair(ax=plt.gca(), scatter_kwargs={'alpha':0.5, 'color':'green'})





In [ ]:
atemp = np.vstack( (samples_diff_avg.samples, samples_list[idx1_pre].samples[-1,:]))
atemp.shape
